In [ ]:
# Exercise 1 #

import ipaddress

def analyse_ip(ip_str, cidr_prefix):
    # Create an IP interface object
    ip_with_cidr = f"{ip_str}/{cidr_prefix}"
    ip = ipaddress.ip_interface(ip_with_cidr)
    network = ip.network

    MASK = '255.255.0.0'

    # host = ipaddress.IPv4Address(ip)
    net = ipaddress.IPv4Network(ip_str + '/' + MASK, False)

    print(f"Address: {ip.ip}")
    print(f"Network: {ip.network}")
    print(f"Netmask: {ip.netmask}")
    print(f"Is private: {ip.ip.is_private}")
    print(f"Is global: {ip.ip.is_global}")
    print(f"Broadcast address: {ip.network.broadcast_address}")
    print(f"CIDR Notation: {network.with_prefixlen}")
    #print(f"First and last usable host addresses: {n[0]}, {n[-1]}")

    # First and last usable host addresses
    all_hosts = list(network.hosts())
    if all_hosts:
        first_host = all_hosts[0]
        last_host = all_hosts[-1]
    else:
        first_host = last_host = "N/A"  # No usable hosts in /31 or /32 networks
    
    print(f"First usable host: {first_host}")
    print(f"Last usable host: {last_host}")
    print(f"Number of usable hosts: {len(all_hosts)}")

    # List all hosts in the network
    if ip.network.num_addresses < 256: # Only for small networks
        print("\nHosts in network:")
        for host in ip.network.hosts():
            print(host)

# Example usage
analyse_ip('192.168.1.1', 24)

Address: 192.168.1.1
Network: 192.168.1.0/24
Netmask: 255.255.255.0
Is private: True
Is global: False
Broadcast address: 192.168.1.255
CIDR Notation: 192.168.1.0/24
First usable host: 192.168.1.1
Last usable host: 192.168.1.254
Number of usable hosts: 254


In [7]:
# Exercise 2 #

import socket
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)

print("Your Computer Name is:" + hostname)
print("Your Computer IP Address is:" + IPAddr)

analyse_ip(IPAddr, 24)

Your Computer Name is:AbrarPC
Your Computer IP Address is:10.100.56.130
Address: 10.100.56.130
Network: 10.100.56.0/24
Netmask: 255.255.255.0
Is private: True
Is global: False
Broadcast address: 10.100.56.255
CIDR Notation: 10.100.56.0/24
First usable host: 10.100.56.1
Last usable host: 10.100.56.254
Number of usable hosts: 254


In [10]:
# Exercise 3 #

import socket

uni_ip = socket.gethostbyname('www.gold.ac.uk')

analyse_ip(uni_ip, 24)


Address: 159.100.136.66
Network: 159.100.136.0/24
Netmask: 255.255.255.0
Is private: False
Is global: True
Broadcast address: 159.100.136.255
CIDR Notation: 159.100.136.0/24
First usable host: 159.100.136.1
Last usable host: 159.100.136.254
Number of usable hosts: 254


In [1]:
# dhcp_simple.py - Simplified DHCP Simulator

# Server Configuration
server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

# Client Configuration
client = {
    "mac": "AA:BB:CC:DD:EE:FF",
    "ip": None
}

def send_discover():
    print("\n[CLIENT] Step 1: Sending DHCP DISCOVER")
    return {
        "type": "DISCOVER",
        "mac": client["mac"]
    }

def make_offer(discover):
    print("\n[SERVER] Step 2: Making DHCP OFFER")
    if not server["ip_pool"]:
        print("No IPs available!")
        return None
    
    offered_ip = server["ip_pool"].pop(0)
    return {
        "type": "OFFER",
        "mac": discover["mac"],
        "ip": offered_ip
    }

def send_request(offer):
    print("\n[CLIENT] Step 3: Sending DHCP REQUEST")
    return {
        "type": "REQUEST",
        "mac": offer["mac"],
        "ip": offer["ip"]
    }

def send_ack(request):
    print("\n[SERVER] Step 4: Sending DHCP ACK")
    server["leases"][request["mac"]] = request["ip"]
    return {
        "type": "ACK",
        "mac": request["mac"],
        "ip": request["ip"]
    }

def main():
    print("=== Simple DHCP Simulation ===")

    # Client starts process
    discover = send_discover()

    # Server responds
    offer = make_offer(discover)
    if not offer:
        return
    
    # Client continues
    request = send_request(offer)

    # Server finalizes
    ack = send_ack(request)

    # Update client IP
    client["ip"] = ack["ip"]

    print("\n=== Result ===")
    print(f"Client {client['mac']} got IP: {client['ip']}")
    print("Server leases:", server["leases"])

if __name__ == "__main__":
    main()


=== Simple DHCP Simulation ===

[CLIENT] Step 1: Sending DHCP DISCOVER

[SERVER] Step 2: Making DHCP OFFER

[CLIENT] Step 3: Sending DHCP REQUEST

[SERVER] Step 4: Sending DHCP ACK

=== Result ===
Client AA:BB:CC:DD:EE:FF got IP: 192.168.1.100
Server leases: {'AA:BB:CC:DD:EE:FF': '192.168.1.100'}


In [ ]:
# Exercise 5 Server #

import socket
import json

# Create a TCP socket
server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server_socket.bind(('localhost', 65432))
server_socket.listen(1) # Allow 1 pending connection

server = {
    "ip_pool": ["192.168.1.100", "192.168.1.101", "192.168.1.102"],
    "leases": {}
}

def make_offer(discover):
    print("\n[SERVER] Step 2: Making DHCP OFFER")
    if not server["ip_pool"]:
        print("No IPs available!")
        return None
    
    offered_ip = server["ip_pool"].pop(0)
    return {
        "type": "OFFER",
        "mac": discover["mac"],
        "ip": offered_ip
    }

def send_ack(request):
    print("\n[SERVER] Step 4: Sending DHCP ACK")
    server["leases"][request["mac"]] = request["ip"]
    return {
        "type": "ACK",
        "mac": request["mac"],
        "ip": request["ip"]
    }


def main():
    print("=== Simple DHCP Simulation ===")

    while True:
        print("\nTCP Server is listening...")
        client_socket, client_address = server_socket.accept()
        
        # Server hears discover
        discover = client_socket.recv(1024)
        discover= json.loads(discover.decode('utf-8'))

        # Server responds
        offer = make_offer(discover)
        if not offer:
            return
        client_socket.sendall(json.dumps(offer).encode('utf-8'))

        # Server finalizes
        request = client_socket.recv(1024)
        request= json.loads(request.decode('utf-8'))
        ack = send_ack(request)
        client_socket.sendall(json.dumps(ack).encode('utf-8'))

        client_socket.close()

        print("\n=== Result ===")
        print("Server leases:", server["leases"])

if __name__ == "__main__":
    main()


=== Simple DHCP Simulation ===

TCP Server is listening...

[SERVER] Step 2: Making DHCP OFFER

[SERVER] Step 4: Sending DHCP ACK

=== Result ===
Server leases: {'AA:BB:CC:DD:EE:FF': '192.168.1.100'}

TCP Server is listening...


#### Problems Encountered:

The dictionaries were not sending at first so I had to use json, specifically the lines 'json.dumps(placeholder).encode('utf-8')' and 'json.loads(placeholder.decode('utf-8'))', to encode and decode the dictionaries to byte data.